In [2]:
import random
import numpy as np
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Board

In [3]:
main_board = [
    ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
    ['W', 'A', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W'],
    ['W', 'D', 'W', 'W', 'W', 'D', 'W', 'W', 'W', 'D', 'W'],
    ['W', 'D', 'W', 'D', 'D', 'D', 'D', 'D', 'W', 'D', 'W'],
    ['W', 'D', 'D', 'D', 'W', 'E', 'W', 'D', 'D', 'D', 'W'],
    ['W', 'D', 'W', 'D', 'W', 'E', 'W', 'D', 'W', 'D', 'W'],
    ['W', 'D', 'W', 'D', 'D', 'W', 'D', 'D', 'W', 'D', 'W'],
    ['W', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W'],
    ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W']
]
board_rows = 7
board_cols = 9

## Visualize

This 2 functions display the board and its changes during ruuning time

In [25]:
WIDTH, HEIGHT = 45, 45
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)

def display_board(screen, map_data):
    for y, row in enumerate(map_data):
        for x, cell in enumerate(row):
            rect = pygame.Rect(x * WIDTH, y * HEIGHT, WIDTH, HEIGHT)
            if cell == 'W':
                pygame.draw.rect(screen, BLUE, rect)
            elif cell == 'A':
                pygame.draw.rect(screen, YELLOW, rect)
            elif cell == 'D':
                pygame.draw.circle(screen, WHITE, (x * WIDTH + WIDTH // 2, y * HEIGHT + HEIGHT // 2), 5)

def display_movements(map_data, reward):
    WIDTH, HEIGHT = 45, 45
    WHITE = (255, 255, 255)
    BLUE = (0, 0, 255)
    YELLOW = (255, 255, 0)
    
    window_size = (WIDTH * map_data.shape[1], HEIGHT * map_data.shape[0])
    screen = pygame.display.set_mode(window_size)
    screen.fill((0, 0, 0))
    
    
    pygame.display.set_caption("Pac-Man Game")

    display_board(screen, main_board)

    
    pygame.display.flip()
    pygame.display.update()
    pygame.time.delay(100)

## Make Q-table

As we considered 3*3 blocks around agent and 3 possible amouts E, W, and D are there, Q-table has $3^8$ rows and $4$ columns.

In [26]:
Q_rows = 3 ** 8
Q_cols = 4
#initialize Q-table
Q = np.zeros((Q_rows, Q_cols))

## Q-function

Define Q-function based on its definition

In [6]:
def Q_function(q, q_max, r, board, alpha = .5, gamma = .5, verbose = 0):
    return ((q + alpha * (r + gamma * q_max - q)))

## Hash function

A function for mapping each state to a row in Q-table. This function converts states to 3-based numbers and map them to the Q-table rows.</br>
it is a sparse matrix and some of its rows(states) could be omitted(as asked in the 1st parth of the question.)
e.g. states which up, left, down, and right of the agent are W, could be omitted from the Q-table to reduce the states.

In [27]:
def hash_function(row, col, board):
    # E -> 0, W -> 1, D -> 2
    # 0 1 2
    # 3 X 4
    # 5 6 7
    matrix = [
        [0, 1, 2],
        [3, 0, 4],
        [5, 6, 7]
    ]
    hash_value = 0
    for i in range(3):
        for j in range(3):
            if i != 1 or j != 1:
                char = board[row + i - 1][col + j - 1]
                if char == 'E':
                    hash_value += 0 * 3**(matrix[i][j])
                elif char == 'W':
                    hash_value += 1 * 3**(matrix[i][j])
                elif char == 'D':
                    hash_value += 2 * 3**(matrix[i][j])
    return hash_value

In [28]:
# a map between hash and state
def get_state(row, col, board):
    return hash_function(row, col, board)

## Reward

Each action has a reward; -2 for wall, -1 for empty, and 5 for dot.

In [29]:
def get_reward(new_state):
    if new_state == 'W':
        return -2
    elif new_state == 'E':
        return -1
    elif new_state == 'D':
        return 5

## Find indices

This function walks through an state and finds the indices of its maximum values.

In [10]:
def find_indices(row, col, state, max_value, board):
    inds = []
    for ind, i in enumerate(state):
        if i == max_value:
            inds.append(ind)
    return inds

## Get action Train

This is a function which finds the indices of max values in a state and with probability epsilon, chooses a random action and with probability (1 - epsilon) chooses an action with max Q-vlaue.

In [31]:
# 0 -> U
# 1 -> R
# 2 -> D
# 3 -> L
def get_action(row, col, state, board, epsilon):
    
    max_value = max(Q[state])
    max_value_indices = find_indices(row, col, Q[state], max_value, board)
    no_max_values = len(max_value_indices)

    choice = random.random()
    
    # choose a max value
    if choice > epsilon:
        return int(random.choice(max_value_indices))
    # choose a random action
    else:
        return random.randint(0, 3)

## Packman

in this section, we try to walk through the map and collect all dots. in each iteration, while there are dots in the board, choose and action, move if it is possible, copmute the new Q-value and update the current state and continue.

In [12]:
import random
def Packman(board, epsilon, verbose=0):
    row = 1
    col = 1
    # count dots in a 2d list
    dots = 0
    for i in range(1, board_rows + 1):
        dots += board[i].count('D')
    
    current_state = get_state(row, col, board)

    while dots > 0:
        
        if verbose > 1: print(f"{row} {col} dots: {dots}", end = ' ')
    
        action = get_action(row, col, current_state, board, epsilon)
        
        reward = 0
        i, j = 0, 0
        if action == 0: # U
            if board[row - 1][col] != 'W': # if the up block is not a wall, move
                board[row][col] = 'E'
                i -= 1
                reward = get_reward(board[row-1][col])
                if verbose > 1: print('U')

        elif action == 1: # R
            if board[row][col + 1] != 'W':
                board[row][col] = 'E'
                j += 1
                reward = get_reward(board[row][col+1])
                if verbose > 1: print('R')

        elif action == 2: # D
            if board[row + 1][col] != 'W':
                board[row][col] = 'E'
                i += 1
                reward = get_reward(board[row+1][col])
                if verbose > 1: print('D')


        elif action == 3: # L
            if board[row][col - 1] != 'W':
                board[row][col] = 'E'
                j -= 1
                reward = get_reward(board[row][col-1])
                if verbose > 1: print('L')
        row += i
        col += j
        
        new_state = current_state
        if board[row][col] == 'D':
            dots -= 1
        if board[row][col] == 'D' or board[row][col] == 'E':
            board[row][col] = 'A'
            new_state = get_state(row, col, board)
            
        elif board[row][col] == 'W':
            reward = get_reward(board[row][col])
            row -= i
            col -= j
            
        

        if dots == 0:
            break
        
        Q[current_state][action] = Q_function(Q[current_state][action], max(Q[new_state]), reward, board, verbose=verbose) # is it right?
        current_state = new_state


# Train

In this section, training is happening. with initial epsilon 0.9, we try to train out model in 200 episodes. each time we have to decrease epsilon as it is the rate of choosing randomly in the traning time.

In [13]:
import time

epsilon = 0.9
rounds  = 200

start = time.time()
for i in range(rounds):
    instance = []
    for j in main_board:
        instance.append(j.copy())
#     print("difed")    
#     print("fff")
    Packman(instance, epsilon)
    epsilon *= (5/9) ** (1/40)
    print(f"{i} iteration")

end = time.time()
print(end - start)    

0 iteration
1 iteration
2 iteration
3 iteration
4 iteration
5 iteration
6 iteration
7 iteration
8 iteration
9 iteration
10 iteration
11 iteration
12 iteration
13 iteration
14 iteration
15 iteration
16 iteration
17 iteration
18 iteration
19 iteration
20 iteration
21 iteration
22 iteration
23 iteration
24 iteration
25 iteration
26 iteration
27 iteration
28 iteration
29 iteration
30 iteration
31 iteration
32 iteration
33 iteration
34 iteration
35 iteration
36 iteration
37 iteration
38 iteration
39 iteration
40 iteration
41 iteration
42 iteration
43 iteration
44 iteration
45 iteration
46 iteration
47 iteration
48 iteration
49 iteration
50 iteration
51 iteration
52 iteration
53 iteration
54 iteration
55 iteration
56 iteration
57 iteration
58 iteration
59 iteration
60 iteration
61 iteration
62 iteration
63 iteration
64 iteration
65 iteration
66 iteration
67 iteration
68 iteration
69 iteration
70 iteration
71 iteration
72 iteration
73 iteration
74 iteration
75 iteration
76 iteration
77 iterat

Print non-zero rows of Q-table as it is asked in the question.

In [14]:
for i in Q:
    flag = False
    for j in i:
        if j!= 0:
            flag = True
    if flag : print(i) 

[ 0.         -0.88257534 -1.          0.        ]
[-1. -1. -1.  0.]
[ 9.24881584 -0.61024669 -0.84240823  3.31372954]
[ 1.45961266e-03  2.50557744e+00 -5.92458574e-01  4.86537552e-04]
[ 0.  0. -1. -1.]
[ 0.          0.55235732 -0.99169049  1.50159061]
[-9.99999990e-001  4.00129104e-199 -1.00000000e+000 -9.96234894e-001]
[ 5.00001759  4.00534885 -0.49999556  0.10559219]
[-1.          0.         -1.49977683  0.        ]
[ 3.8429786   1.235866   -1.20038661  1.88208404]
[ 2.93447181  0.         -0.94277748  0.        ]
[ 2.99875193  0.         -0.73870826  0.        ]
[ 0.          0.         -1.31228638  0.        ]
[ 9.92789345  3.62553373 -1.43995734  4.69301621]
[ 3.7139181   1.97077019 -0.26270473  8.00757098]
[ 0.2234812   1.2487793  -0.03749368  4.99998093]
[8.1080962 0.        0.        0.       ]
[ 0.          2.5        -0.25390625  0.        ]
[-0.25003186  5.         -0.08218365  2.41735129]
[5.89904563 5.36697937 0.18617138 2.50372119]
[ 2.97803317  7.5001496  -0.17117747  3.

This function is for choosing action in testing time.
It always chooses max values among all possible actions.

In [15]:
def get_action_test(row, col, state, board):
        
    max_value = max(Q[state])
    max_value_indices = find_indices(row, col, Q[state], max_value, board)
    no_max_values = len(max_value_indices)
# ----------------------------------------------------
#     choice = random.random()
#     if choice < .1:
#         return random.randint(0, 3)
# ----------------------------------------------------

    return random.choice(max_value_indices)

this function also handles the testing part, choosing action, moving, and going to the next state

In [21]:
def test_Packman(board, dots, verbose=2):
    row = 1
    col = 1

    pre = 0
    cnt = 0
    
    total_reward = 0
    
    current_state = get_state(row, col, board)
    iteration = 0
    action = None
    pygame.init()
    display_movements(board, total_reward)
    while dots > 0:
        print("total reward: ", total_reward)
        if verbose > 1: print(f"{row} {col} dots: {dots}", end = ' ')
        
        if cnt > 5:
            action = random.randint(0, 3)
            cnt = 0
        else:
            action = get_action_test(row, col, current_state, board)
        
        i,j = 0, 0
        if action == 0: # U
            if board[row - 1][col] != 'W':
                board[row][col] = 'E'
                i -= 1
                if verbose > 1: print('U')

        elif action == 1: # R
            if board[row][col + 1] != 'W':
                board[row][col] = 'E'
                j += 1

                if verbose > 1: print('R')

        elif action == 2: # D
            if board[row + 1][col] != 'W':
                board[row][col] = 'E'
                i += 1
                if verbose > 1: print('D')

        elif action == 3: # L
            if board[row][col - 1] != 'W':
                board[row][col] = 'E'
                j -= 1
                if verbose > 1: print('L')
        
        row += i
        col += j
        
        if board[row][col] == 'D':
            total_reward += 5
            dots -= 1
            cnt = 0
            board[row][col] = 'A'
        elif board[row][col] == 'E':
            total_reward -= 1
            board[row][col] = 'A'
        if board[row][col] != 'D':
            cnt += 1

        

        iteration += 1
        
        print("iterations: ", iteration)
        print('-' * 60)
    
        
        current_state = get_state(row, col, board)
        pre = current_state
        
        display_movements(board, total_reward)
    pygame.quit()

Test the code with the main board: it took 54 iterations to eat all 43 dots!

In [24]:
test_1 = []
dots = 0
for row in main_board:
    dots += row.count('D')
    test_1.append(row.copy())
test_Packman(np.array(test_1), dots)

total reward:  0
1 1 dots: 43 D
iterations:  1
------------------------------------------------------------
total reward:  5
2 1 dots: 42 D
iterations:  2
------------------------------------------------------------
total reward:  10
3 1 dots: 41 D
iterations:  3
------------------------------------------------------------
total reward:  15
4 1 dots: 40 D
iterations:  4
------------------------------------------------------------
total reward:  20
5 1 dots: 39 D
iterations:  5
------------------------------------------------------------
total reward:  25
6 1 dots: 38 D
iterations:  6
------------------------------------------------------------
total reward:  30
7 1 dots: 37 R
iterations:  7
------------------------------------------------------------
total reward:  35
7 2 dots: 36 R
iterations:  8
------------------------------------------------------------
total reward:  40
7 3 dots: 35 R
iterations:  9
------------------------------------------------------------
total reward:  45
7 4

take another board for testing the model

In [22]:
board_4 = [
    ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
    ['W', 'A', 'D', 'D', 'D', 'D', 'D', 'W'],
    ['W', 'D', 'W', 'W', 'D', 'W', 'D', 'W'],
    ['W', 'D', 'W', 'D', 'D', 'D', 'D', 'W'],
    ['W', 'D', 'W', 'W', 'E', 'W', 'D', 'W'],
    ['W', 'D', 'D', 'D', 'D', 'D', 'D', 'W'],
    ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W']
]

test_4 = []
dot_4 = 0
for i in board_4:
    dot_4 += i.count('D')
    test_4.append(i.copy())
print("here: ", dot_4)
test_Packman(np.array(test_4), dot_4)

here:  21
total reward:  0
1 1 dots: 21 D
iterations:  1
------------------------------------------------------------
total reward:  5
2 1 dots: 20 D
iterations:  2
------------------------------------------------------------
total reward:  10
3 1 dots: 19 D
iterations:  3
------------------------------------------------------------
total reward:  15
4 1 dots: 18 D
iterations:  4
------------------------------------------------------------
total reward:  20
5 1 dots: 17 R
iterations:  5
------------------------------------------------------------
total reward:  25
5 2 dots: 16 iterations:  6
------------------------------------------------------------
total reward:  25
5 2 dots: 16 iterations:  7
------------------------------------------------------------
total reward:  25
5 2 dots: 16 L
iterations:  8
------------------------------------------------------------
total reward:  24
5 1 dots: 16 iterations:  9
------------------------------------------------------------
total reward:  24